<a href="https://colab.research.google.com/github/donotcallS/Bigdata_Training_at_ITwill/blob/main/Team_Project/crawling_BS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
#> 종목 코드
code = {'삼성전자':'005930', 'LG전자':'066570', 'JYP_Ent':'035900', 'YG_ENt':'122870'}

In [185]:
#> 함수(종목코드, 시작페이지, 종료페이지, 저장할 파일이름)
def crawling_news(code_num, start_page_num=1, end_page_num=2, file_name='test.csv'):

    # title, 본문url selector + 관련 뉴스의 title, 본문url
    title_selector = 'body > div > table.type5 > tbody > tr > td.title > a, tr.relation_lst > td > table > tbody > tr > td.title > a'
    # time selector + 관련 뉴스의 time
    time_selector = 'body > div > table.type5 > tbody > tr > td.date, tr.relation_lst > td > table > tbody > tr > td.date'
    # 본문 내용 selector
    content_selector = 'div#news_read'

    # 데이터 저장
    data = {}
    data_titles = []
    data_address = []
    data_times = []
    data_contents = []
    break_flag = False      #> 마지막에 더이상 뉴스가 없을때 마지막 뉴스가 중복되어서 나옴, 그럴 때 종료를 하기 위한 flag

    # start 부터 end 까지 반복문
    for page_num in range(start_page_num,end_page_num+1):
        url = f'https://finance.naver.com/item/news_news.nhn?code={code_num}&page={page_num}'
        res = requests.get(url)
        html = res.text
        soup = BeautifulSoup(html, 'lxml')
        #soup = BeautifulSoup(html, 'html.parser')
        
        titles = soup.select(title_selector)
        times = soup.select(time_selector)

        for title in titles:
            # title, address 추가
            if len(data_address) != 0:  #> 처음 시작에는 종료가 발생하지 않도록
                if data_titles[len(data_titles)-1] == title.text: #> 이전 뉴스의 title과 같은 title 이면
                    break_flag = True   # 종료 flag를 바꿔주고 for문 break
                    break
            data_titles.append(title.text) 
            data_address.append('https://finance.naver.com/'+title.get('href'))
            
            # contents 추가
            url = 'https://finance.naver.com/'+title.get('href') #> 저장한 본문 url 불러오기
            res = requests.get(url)
            html = res.text
            soup = BeautifulSoup(html, 'lxml')
            #soup = BeautifulSoup(html, 'html.parser')
            contents = soup.select(content_selector)
            for content in contents:
                data_contents.append(content.text.split(sep='@')[0])
        
        if break_flag == True:  #> 종료 flag가 on이면 종료 == 마지막뉴스까지 긁었다.
            print('마지막 뉴스 => 종료')
            break

        for time in times:
            data_times.append(time.text)

        data['title'] = data_titles
        data['time'] = data_times
        data['address'] = data_address
        data['content'] = data_contents

    #> dataframe 형식으로 저장
    result = pd.DataFrame(data)
    
    # 시간순으로 sort 하고 덮어쓰기
    result.sort_values(by=['time'], axis=0, ascending=False, inplace=True)

    # 중복된 기사들 있어서 추가, title이 중복되면 가장 최신(시간순sort)의 기사 제외 삭제, 수정사항 덮어쓰기
    result.drop_duplicates(['title'], keep='first', inplace=True)

    # CSV 파일로 저장
    result.to_csv(file_name, sep=',', na_rep='NaN')  

In [182]:
crawling_news(code['JYP_Ent'], 16, 20, 'JYP.csv')

종료


In [183]:
test = pd.read_csv('/content/JYP.csv')

In [187]:
len(test)

19

In [188]:
test

,Unnamed: 0,title,time,address,content
0,0,[fnRASSI]JYP Ent.(035900) 전일대비 11.01% 상...,2020.05.19 11:05,https://finance.naver.com//item/news_read.nhn?...,"\n현재 JYP Ent.(035900)는 전일대비 11.01% 상승하여 23,700..."
1,1,"'JYP Ent.' 10% 이상 상승, 기대 이상의 1분기 실적 - 현...",2020.05.19 10:59,https://finance.naver.com//item/news_read.nhn?...,"\n◆ 최근 애널리스트 분석의견- 기대 이상의 1분기 실적 - 현대차증권, BUY..."
2,2,"JYP, IP의 힘…타사보다 적은 코로나19 타격-KTB",2020.05.19 08:52,https://finance.naver.com//item/news_read.nhn?...,\n\t\t\t [머니투데이 김태현 기자] KTB투자증권은 JYP Ent.에 대해 ...
3,10,"""JYP Ent., 코로나19로 인한 실적타격 가장 적어""-KTB투자...",2020.05.19 08:29,https://finance.naver.com//item/news_read.nhn?...,\n\t\t\t [파이낸셜뉴스] KTB투자증권은 19일 JYP Ent.에 대해 ...
4,3,"JYP카페, 롯데월드몰에 둥지 틀었다",2020.05.15 11:19,https://finance.naver.com//item/news_read.nhn?...,\n300㎡규모 체험카페…신사옥 외 첫 출점유기농 즐기고 이벤트 참여…굿즈 구입도롯...
5,11,롯데월드몰에 ‘JYP 카페’ 생겼다,2020.05.15 09:59,https://finance.naver.com//item/news_read.nhn?...,\n롯데월드몰 ‘JYP 소울컵’ 매장 내 설치된 ‘무인 오디션 스튜디오 부스’ [파...
6,12,"JYP 카페, 롯데월드몰에 둥지 틀었다",2020.05.15 09:54,https://finance.naver.com//item/news_read.nhn?...,"\nJYP소울컵, 롯데월드몰 출점… 신사옥 외 처음 유기농 메뉴 즐기고 이벤트 참여..."
7,13,"롯데월드몰, ‘JYP 카페’ 국내 두 번째 지점 오픈",2020.05.15 09:07,https://finance.naver.com//item/news_read.nhn?...,\n90평 규모 'JYP소울컵' 롯데월드몰 2층에 열어JYP엔터테인먼트 아티스트와 ...
8,4,"JYP엔터, 3분기까지 코로나19 악영향 불가피-NH",2020.05.15 08:51,https://finance.naver.com//item/news_read.nhn?...,\n\t\t\t [이데일리 유준하 기자] NH투자증권은 15일 JYP Ent.(03...
9,5,"JYP엔터테인먼트, 코로나로 영업익 500억 달성 힘들듯-하나금투",2020.05.15 08:45,https://finance.naver.com//item/news_read.nhn?...,\n\t\t\t [머니투데이 강민수 기자] 하나금융투자는 15일 JYP Ent.에 ...
